In [ ]:
import os
import glob
import pandas as pd
import json
import numpy as np

import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN

from transformers import BertTokenizer, BertModel
import torch

import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

import pandas as pd

import re
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from pycaret.classification import setup, compare_models, evaluate_model, predict_model, finalize_model

import numpy as np

In [ ]:
df1 = pd.read_csv('./dataset_1.csv', index_col = 0)
df2 = pd.read_csv(',/dataset_2.csv')
df2 = df2.drop_duplicates()
data1 = df1[['keyword', 'label']].rename(columns = {'keyword' : 'content'})

In [ ]:
label_counts = df['label'].value_counts()
min_count = label_counts.min()
sampling_count = min_count * 25

def sample_data(group):
    return group.sample(n=min(len(group), sampling_count))

data = df.groupby('label', group_keys=False).apply(sample_data)
data['label'].value_counts()
data.reset_index(drop = True, inplace = True)

In [ ]:

def preprocess_text(text):
    if isinstance(text, str):
        text = re.sub(r'\[.*?\]', '', text)  
        text = re.sub(r'\W+', ' ', text)  
        text = re.sub(r'\s+', ' ', text).strip()  
    else:
        text = ''
    return text


data['content'] = data['content'].apply(preprocess_text)

# 텍스트와 라벨
texts = data['content'].tolist()
labels = data['label'].tolist()

# 라벨 인코딩
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# 데이터 오버샘플링
ros = RandomOverSampler()
texts_resampled, labels_resampled = ros.fit_resample(pd.DataFrame(texts, columns=['text']), pd.Series(labels_encoded))

# 데이터셋 분할
train_texts, val_texts, train_labels, val_labels = train_test_split(texts_resampled['text'], labels_resampled, test_size=0.2, stratify=labels_resampled)

# KoBERT 토크나이저 초기화
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')

# 데이터셋 전처리 함수
def convert_data_to_examples(texts, labels):
    input_examples = []
    for text, label in zip(texts, labels):
        input_examples.append(InputExample(guid=None, text_a=text, text_b=None, label=label))
    return input_examples

train_examples = convert_data_to_examples(train_texts, train_labels)
val_examples = convert_data_to_examples(val_texts, val_labels)

# 입력 포맷 변환
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = []
    for e in examples:
        input_dict = tokenizer.encode_plus(e.text_a,
                                           add_special_tokens=True,
                                           max_length=max_length,
                                           return_token_type_ids=True,
                                           return_attention_mask=True,
                                           padding='max_length',
                                           truncation=True)

        input_ids, token_type_ids, attention_mask = input_dict['input_ids'], input_dict['token_type_ids'], input_dict['attention_mask']
        features.append(
            InputFeatures(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label))

    def gen():
        for f in features:
            yield (
                {
                    'input_ids': f.input_ids,
                    'attention_mask': f.attention_mask,
                    'token_type_ids': f.token_type_ids
                }, f.label
            )

    return tf.data.Dataset.from_generator(gen,
        ({
            'input_ids': tf.int32,
            'attention_mask': tf.int32,
            'token_type_ids': tf.int32
        }, tf.int64),
        ({
            'input_ids': tf.TensorShape([None]),
            'attention_mask': tf.TensorShape([None]),
            'token_type_ids': tf.TensorShape([None])
        }, tf.TensorShape([]))
    )

train_dataset = convert_examples_to_tf_dataset(train_examples, tokenizer)
val_dataset = convert_examples_to_tf_dataset(val_examples, tokenizer)

train_dataset = train_dataset.shuffle(100).batch(16).repeat(2)
val_dataset = val_dataset.batch(16)

# 모델 초기화
model = TFBertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=len(label_encoder.classes_))

# 모델 컴파일
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

model.fit(train_dataset, epochs=10, validation_data=val_dataset)


model.save_pretrained('/newsurfing/kobert_news_classifier')
tokenizer.save_pretrained('/newsurfing/kobert_news_classifier')

np.save('/newsurfing/kobert_news_classifier/label_classes.npy', label_encoder.classes_)